# Generate font size data visualization

In [1]:
import sys
print(sys.version)
import pandas as pd
import nltk
%matplotlib inline
import math
import json
from ast import literal_eval
import numpy as np
import re
from matplotlib import pyplot as plt
from colour import Color
from IPython.core.display import HTML
from matplotlib import cm
from matplotlib.colors import rgb2hex
plt.rcParams["figure.figsize"] = [16, 6]

3.10.9 (main, Mar  1 2023, 12:33:47) [Clang 14.0.6 ]


In [2]:
# ACTION: specify top-level data directory and project info here
# Make sure they match the info you specified in preprocessing

dataDir = "/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data"
authorSurname = "Woolf"
publicationYear = "1925"
textTitle = "Dalloway"

In [3]:
# Load variables
projectName = f"{authorSurname}_{publicationYear}_{textTitle}"
sourceDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Source"
corpusDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Corpus"
resultsDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Results"

In [4]:
resultsData = f"{resultsDir}/{projectName}_results.jsonl"

with open(resultsData) as f: 
    rawMatches = f.readlines()

data = [json.loads(line) for line in rawMatches]
df = pd.DataFrame(data)

In [5]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [6]:
sourceText = f"{sourceDir}/{projectName}_plaintext.txt"

text = Text(sourceText)

In [7]:
# Get the size of the text. 
textALength = text.length

# Make an empty array the size of the text. 
tally = np.zeros(textALength, dtype=np.int)

/var/folders/7r/pgtddj0504z5s_9j2nqdrx0r0000gn/T/ipykernel_48195/3521959632.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tally = np.zeros(textALength, dtype=np.int)


In [8]:
# Optional: drop null cells for incomplete runs of matcher
# df = df.dropna(subset='Locations in A')
# df

In [9]:
# Read the matched locations from the results dataset, and literally evaluate them into lists. 

locationsInA = df['Locations in A']

In [10]:
# Tally up every time a letter in the text is quoted. 
for article in locationsInA: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [11]:
# Make a color list in hex for all the values in the tally. 
# Let's hope there aren't too many. 
colors = list(np.arange(5,75,70/(tally.max()+1)))
colorList = colors

In [12]:
# Create a CSS Stylesheet for each color value in the map. 
colorCSS = ""
for i, color in zip(range(0, tally.max()+1), colorList): 
    colorCSS += ".c-%s { font-size: %spx; }\n" % (i, color)

In [13]:
n = 50

checkpoints = np.linspace(0, textALength, n).round()
checkpoints = [int(point) for point in checkpoints]

In [14]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, text.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    if i in checkpoints: 
        out += '<a name="b-%s"></a>' % checkpoints.index(i)
    out += char
    previousVal = val

In [15]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  pre { 
      font-family: Raleway, serif; 
    }
  main { 
      width: 40em; 
      margin: 2em auto; 
  }
  %s
  </style>
  """ % (colorCSS)
html += """
  </head>
  <body><main><pre>%s</pre></main></body></html>
  """ % (out)

In [16]:
with open(f"{resultsDir}/{projectName}_Annotated_by_Size.html", 'w') as f: 
    f.write(html)
    f.close()